In [2]:
import pandas as pd

In [5]:
df = pd.read_csv('deepsolar_tract.csv', encoding='latin-1')

In [6]:
df.columns

Index(['Unnamed: 0', 'tile_count', 'solar_system_count', 'total_panel_area',
       'fips', 'average_household_income', 'county', 'education_bachelor',
       'education_college', 'education_doctoral',
       ...
       'incentive_count_nonresidential', 'incentive_residential_state_level',
       'incentive_nonresidential_state_level', 'net_metering', 'feedin_tariff',
       'cooperate_tax', 'property_tax', 'sales_tax', 'rebate',
       'avg_electricity_retail_rate'],
      dtype='object', length=169)